<a href="https://colab.research.google.com/github/Claptar/diploma/blob/main/DE/human_young_vs_old_scanpy/L2_young_vs_old.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Dependencies and files

## Install dependencies

In [ ]:
!pip install scanpy[leiden] anndata2ri scikit-misc scvi-tools squidpy gseapy==0.10.8 --quiet

# Необходимо, чтобы конкретно прогрузился matplotlib
import os
os.kill(os.getpid(), 9)

## Imports

In [51]:
import warnings
import scanpy as sc
import squidpy as sq
import anndata as an
import gseapy
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from urllib import request
import json
import os

sc.settings.set_figure_params(dpi=80)
#sc.set_figure_params(facecolor="white", figsize=(8, 8))
warnings.simplefilter(action='ignore', category=FutureWarning)
sc.settings.verbosity = 3

# Global variables

Нужно подключить свой гугл диск

In [52]:
dir_path = 'drive/MyDrive/Spatial project/results/'

In [53]:
os.listdir('drive/MyDrive/Spatial project/results/filtered_samples/')

['human.h5ad', 'chimp.h5ad', 'macaque.h5ad', 'spatial_libd_human.h5ad']

In [54]:
# read young human adata object
human_young = sc.read_h5ad(dir_path + 'filtered_samples/human.h5ad')
print(human_young)

# add _young sufix to layer name
human_young.obs.label.replace({"L6a": "L6", "L6b": "L6"}, inplace=True)
human_young.obs.label = human_young.obs.label.astype('str') + '_young'

AnnData object with n_obs × n_vars = 15417 × 19966
    obs: 'in_tissue', 'array_row', 'array_col', 'label', 'sample_id', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'total_counts_hb', 'pct_counts_hb', 'total_counts_ribo', 'pct_counts_ribo'
    var: 'gene_ids', 'feature_types', 'mt', 'hb', 'ribo'
    uns: 'qc_good_spots_colors', 'qc_n_genes_by_counts_colors', 'qc_pct_counts_ribo_colors', 'qc_total_counts_colors', 'spatial'
    obsm: 'spatial'


/usr/local/lib/python3.9/dist-packages/anndata/_core/anndata.py:1828: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")


In [55]:
# read young human adata object
human_old = sc.read_h5ad(dir_path + 'filtered_samples/spatial_libd_human.h5ad')
print(human_old)

# add _young sufix to layer name
human_old.obs.label = human_old.obs.label.astype('str') + '_old'

AnnData object with n_obs × n_vars = 42588 × 33538
    obs: 'in_tissue', 'array_row', 'array_col', 'label', 'sample_id', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'total_counts_hb', 'pct_counts_hb', 'total_counts_ribo', 'pct_counts_ribo'
    var: 'gene_ids', 'feature_types', 'genome', 'mt', 'hb', 'ribo'
    uns: 'qc_good_spots_colors', 'qc_n_genes_by_counts_colors', 'qc_pct_counts_mt_colors', 'qc_pct_counts_ribo_colors', 'qc_total_counts_colors', 'spatial'
    obsm: 'spatial'


/usr/local/lib/python3.9/dist-packages/anndata/_core/anndata.py:1828: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")


In [ ]:
# concat old and young files
adata = an.concat([human_young, human_old], merge='same',uns_merge="unique")
adata.obs_names_make_unique()
adata

In [ ]:
samples = list(adata.uns['spatial'].keys())
samples

In [ ]:
del human_old, human_young

# Посмотрим на картинки

In [ ]:
sq.pl.spatial_scatter(adata, color=['label'], library_key='sample_id', size=1.1, ncols=5, title=samples)

# Differential expression

In [ ]:
key = 'DE_L3'
groups = ["L3_young", "L3_old"]

In [ ]:
sc.pp.normalize_total(adata, target_sum=1e4)
sc.pp.log1p(adata)

In [ ]:
de_adata = adata[adata.obs.label.isin(groups)]
de_adata

## Feature selection

In [ ]:
sc.pp.highly_variable_genes(
    adata=de_adata, flavor="seurat", 
    n_top_genes=3000, inplace=True,
    batch_key="sample_id")

In [ ]:
highly_variable_genes = de_adata.var[de_adata.var.highly_variable].index.tolist()

## PCA

In [ ]:
sc.tl.pca(de_adata)

In [ ]:
sc.pl.pca(de_adata, color=['sample_id'], components = ['1,2','3,4','5,6','7,8'], ncols=2)

In [ ]:
sc.pl.pca_loadings(de_adata, components=[1,2,3,4,5,6,7,8])

Уберем митохондриальные и рибосомные гены

In [ ]:
mito = de_adata.var_names.str.startswith('MT-')
hb = de_adata.var_names.str.contains(("^HB[AB]"))
rb = de_adata.var_names.str.contains(("^RP[LS]"))
remove = mito | hb | rb
genes_to_remove = de_adata.var_names[remove].tolist()

keep = np.invert(remove)
print(sum(remove))

de_adata = de_adata[:,keep]

In [ ]:
# remove genes from `highly_variable_genes`
highly_variable_genes = [gene for gene in highly_variable_genes if gene not in genes_to_remove]

## Rank genes

In [ ]:
sc.tl.rank_genes_groups(de_adata, groupby='label', method='wilcoxon', pts=True)
adata.uns[key] = de_adata.uns["rank_genes_groups"]

In [ ]:
sc.pl.rank_genes_groups(de_adata, n_genes=25, sharey=False)

**TUBA1A** — Microtubules of the eukaryotic cytoskeleton perform essential and diverse functions and are composed of a heterodimer of alpha and beta tubulins

**STMN2** — This gene encodes a member of the stathmin family of phosphoproteins. Stathmin proteins function in microtubule dynamics and signal transduction. The encoded protein plays a regulatory role in neuronal growth and is also thought to be involved in osteogenesis. Reductions in the expression of this gene have been associated with Down's syndrome and Alzheimer's disease.

**YBX1** — This gene encodes a highly conserved cold shock domain protein that has broad nucleic acid binding properties

**SNAP25** — Synaptic vesicle membrane docking and fusion is mediated by SNAREs (soluble N-ethylmaleimide-sensitive factor attachment protein receptors) located on the vesicle membrane (v-SNAREs) and the target membrane (t-SNAREs)

**CLU** — The protein encoded by this gene is a secreted chaperone that can under some stress conditions also be found in the cell cytosol. It has been suggested to be involved in several basic biological events such as cell death, tumor progression, and neurodegenerative disorders.

**MT3** — This gene is a member of the metallothionein family of genes. Proteins encoded by this gene family are low in molecular weight, are cysteine-rich, lack aromatic residues, and bind divalent heavy metal ions. This gene family member displays tissue-specific expression, and contains a threonine insert near its N-terminus and a glutamate-rich hexapeptide insert near its C-terminus relative to the proteins encoded by other gene family members.

## Посмотрим на дифф. экспрессированые гены

In [ ]:
genes_rank_df = sc.get.rank_genes_groups_df(de_adata, group=groups[0])
genes_rank_df.index = genes_rank_df.names
genes_rank_df['abs_lfc'] = genes_rank_df['logfoldchanges'].abs()
genes_rank_df['abs_scores'] = genes_rank_df['scores'].abs()
genes_rank_df

### Распределение p-value

In [ ]:
genes_rank_df.loc[highly_variable_genes, 'pvals_adj'].hist(bins=50, grid=False)

Не понимаю на сколько нормально видеть такую картину

### Сравним молодые/старые

In [ ]:
top_genes_df = genes_rank_df.loc[highly_variable_genes][(genes_rank_df.pvals_adj < 0.05) &
                                                        (genes_rank_df.pct_nz_group > 0.1) &
                                                        (genes_rank_df.pct_nz_reference > 0.1)].sort_values(by='abs_lfc', ascending=False)
top_genes_df.head()

In [ ]:
genes1 = genes_rank_df.index[:10].tolist()
genes2 = top_genes_df.index[:10].tolist()
sc.pl.rank_genes_groups_violin(de_adata, gene_names=genes1, groups=groups[0])
sc.pl.rank_genes_groups_violin(de_adata, gene_names=genes2, groups=groups[0])

In [ ]:
sc.pl.rank_genes_groups_dotplot(de_adata, groupby="label", n_genes=10, values_to_plot='logfoldchanges',
                                min_logfoldchange=1, vmax=3, vmin=-3, cmap='bwr')

### Посмотрим на отдельные образцы

In [ ]:
sc.pl.violin(de_adata, genes1[:5], groupby='sample_id', rotation=90)
sc.pl.violin(de_adata, genes2[:5], groupby='sample_id', rotation=90)

In [ ]:
sc.pl.dotplot(de_adata, genes1 + genes2, groupby='sample_id')

In [ ]:
sc.pl.rank_genes_groups_dotplot(de_adata, groupby="sample_id", n_genes=10)

### Сохраним csv файл

In [ ]:
genes_rank_df = sc.get.rank_genes_groups_df(de_adata, group=groups[0])
genes_rank_df.to_csv(dir_path + 'DE/young_vs_old_scanpy/' + key + '.csv')

## Gene Set Analysis

In [ ]:
gene_set_names = gseapy.get_library_name(organism='Human')
gene_set_names[:5]
#gene_set_names

Выберем только дифференциально экспрессированые гены

In [ ]:
gene_set = sc.get.rank_genes_groups_df(de_adata, group=groups[0], log2fc_min=0.25, pval_cutoff=0.05).set_index('names').loc[de_adata.var["highly_variable"]]
gene_set = gene_set[(gene_set.pct_nz_group > 0.1) & (gene_set.pct_nz_reference > 0.2)]
gene_list = gene_set.index.str.strip().tolist()
print(len(gene_list))

Возьмём базу данных `GO_Biological_Process_2021`

In [ ]:
enr_res = gseapy.enrichr(gene_list=gene_list,
                     organism='Human',
                     gene_sets='GO_Biological_Process_2021',
                     cutoff = 0.5)

In [ ]:
gseapy.barplot(enr_res.res2d, title='GO_Biological_Process_2021')

## GSEA

### Scores

In [ ]:
scores = (
    sc.get.rank_genes_groups_df(de_adata, group=groups[0])
    .set_index('names')
    .loc[de_adata.var["highly_variable"]]
    .sort_values("scores", ascending=False)
    [['scores']]
    .rename_axis(["young_L1"], axis=1)
    )
scores

In [ ]:
res = gseapy.prerank(rnk=scores, gene_sets='KEGG_2021_Human')

In [ ]:
res.res2d.head()

In [ ]:
terms = res.res2d.index
terms[:20]

In [ ]:
gseapy.gseaplot(rank_metric=res.ranking, term=terms[15], **res.results[terms[15]])

In [ ]:
gseapy.gseaplot(rank_metric=res.ranking, term=terms[2], **res.results[terms[2]])